#Load and manipulate the dataset

In [1]:
import pandas as pd

# Load the MedQA dataset
dataset = pd.read_csv('train.csv')


In [2]:
dataset.head()


,qtype,question,answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [3]:
dataset = pd.read_csv('train.csv')

# Convert to pandas DataFrame
df = pd.DataFrame(dataset)

# Display the first few rows of the dataframe
print(df.head())

# Select necessary columns (question and answer)
df = df[['question', 'answer']]

# Drop rows with missing values if any
df.dropna(inplace=True)

# Display the updated dataframe
print(df.head())

# Ensure we have 10,000 samples
df = df.sample(n=10000, random_state=42)

# Reset the index
df.reset_index(drop=True, inplace=True)


             qtype                                           question  \
0   susceptibility  Who is at risk for Lymphocytic Choriomeningiti...   
1         symptoms  What are the symptoms of Lymphocytic Choriomen...   
2   susceptibility  Who is at risk for Lymphocytic Choriomeningiti...   
3  exams and tests  How to diagnose Lymphocytic Choriomeningitis (...   
4        treatment  What are the treatments for Lymphocytic Chorio...   

                                              answer  
0  LCMV infections can occur after exposure to fr...  
1  LCMV is most commonly recognized as causing ne...  
2  Individuals of all ages who come into contact ...  
3  During the first phase of the disease, the mos...  
4  Aseptic meningitis, encephalitis, or meningoen...  
                                            question  \
0  Who is at risk for Lymphocytic Choriomeningiti...   
1  What are the symptoms of Lymphocytic Choriomen...   
2  Who is at risk for Lymphocytic Choriomeningiti...   
3  How 

#Prepare the prompts

#Define the prompt templates for zero-shot, one-shot, and few-shot prompting.

In [4]:
def create_zero_shot_prompt(question):
    return f"Q: {question}\nA:"

def create_one_shot_prompt(question, example):
    return f"Example:\nQ: {example['question']}\nA: {example['answer']}\n\nQ: {question}\nA:"

def create_few_shot_prompt(question, examples):
    prompt = "Examples:\n"
    for _, ex in examples.iterrows():
        prompt += f"Q: {ex['question']}\nA: {ex['answer']}\n\n"
    prompt += f"Q: {question}\nA:"
    return prompt


In [5]:
!pip install openai

#Evaluate the models

In [7]:
import openai

# Set your OpenAI API key
openai.api_key = 'sk-proj-bOjdc6DKPUnL4eIGq7v8T3BlbkFJVtGfF4dJsSXUMxozOXgV'

def get_response(model, prompt):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=150
    )
    return response.choices[0].message['content'].strip()

# Define a function to evaluate the models
def evaluate_model(model_name, df, prompt_type, examples_df=None):
    correct = 0
    for _, row in df.iterrows():
        if prompt_type == 'zero_shot':
            prompt = create_zero_shot_prompt(row['question'])
        elif prompt_type == 'one_shot':
            prompt = create_one_shot_prompt(row['question'], examples_df.iloc[0])
        elif prompt_type == 'few_shot':
            prompt = create_few_shot_prompt(row['question'], examples_df)
        
        response = get_response(model_name, prompt)
        if response.lower() == row['answer'].lower():
            correct += 1
    
    accuracy = correct / len(df)
    return accuracy

# Randomly sample examples for one-shot and few-shot prompts
example_df = df.sample(n=5, random_state=42)





#Compare the models

In [8]:
# Evaluate GPT-3.5
accuracy_gpt35_zero_shot = evaluate_model("gpt-3.5-turbo-instruct", df, 'zero_shot')
accuracy_gpt35_one_shot = evaluate_model("gpt-3.5-turbo-instruct", df, 'one_shot', example_df.iloc[:1])
accuracy_gpt35_few_shot = evaluate_model("gpt-3.5-turbo-instruct", df, 'few_shot', example_df)

# Evaluate GPT-4
accuracy_gpt4_zero_shot = evaluate_model("gpt-4", df, 'zero_shot')
accuracy_gpt4_one_shot = evaluate_model("gpt-4", df, 'one_shot', example_df.iloc[:1])
accuracy_gpt4_few_shot = evaluate_model("gpt-4", df, 'few_shot', example_df)

# Print the results
print(f"GPT-3.5 Zero-shot accuracy: {accuracy_gpt35_zero_shot}")
print(f"GPT-3.5 One-shot accuracy: {accuracy_gpt35_one_shot}")
print(f"GPT-3.5 Few-shot accuracy: {accuracy_gpt35_few_shot}")

print(f"GPT-4 Zero-shot accuracy: {accuracy_gpt4_zero_shot}")
print(f"GPT-4 One-shot accuracy: {accuracy_gpt4_one_shot}")
print(f"GPT-4 Few-shot accuracy: {accuracy_gpt4_few_shot}")




RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.